In [1]:
import numpy as np
import pandas as pd
 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib 

In [2]:
traindata = pd.read_csv('D:\KULIAH\Semester 8\Dataset\preprocessed_train_4_new.csv')
testdata = pd.read_csv('D:\KULIAH\Semester 8\Dataset\preprocessed_test_4_new.csv')

traindata.pop("Unnamed: 0")
testdata.pop("Unnamed: 0")

traindata.head()

,URG Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,Pkt Size Avg,Flow Pkts/s,FIN Flag Cnt,ECE Flag Cnt,ACK Flag Cnt,Dst Port,Label,Protocol_0,Protocol_6,Protocol_17
0,0,0,0,1,225.900000,0.441997,0,0,0,443,0,0,1,0
1,0,0,0,0,78.500000,7220.216606,0,0,0,53,0,0,0,1
2,0,0,0,0,164.666667,1.006194,0,0,1,443,0,0,1,0
3,0,0,0,0,49.500000,4.721926,0,0,0,5355,0,0,0,1
4,0,0,0,0,0.000000,5221.932115,0,0,1,80,0,0,1,0


In [3]:
Y = traindata.pop('Label')
X = traindata.iloc[:,0:13]
C = testdata.pop('Label')
T = testdata.iloc[:,0:13]

X.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 832438 entries, 0 to 832437
Data columns (total 13 columns):
URG Flag Cnt    832438 non-null int64
SYN Flag Cnt    832438 non-null int64
RST Flag Cnt    832438 non-null int64
PSH Flag Cnt    832438 non-null int64
Pkt Size Avg    832438 non-null float64
Flow Pkts/s     832438 non-null float64
FIN Flag Cnt    832438 non-null int64
ECE Flag Cnt    832438 non-null int64
ACK Flag Cnt    832438 non-null int64
Dst Port        832438 non-null int64
Protocol_0      832438 non-null int64
Protocol_6      832438 non-null int64
Protocol_17     832438 non-null int64
dtypes: float64(2), int64(11)
memory usage: 82.6 MB


In [4]:
_traindata = np.array(X)
_trainlabel = np.array(Y)

_testdata = np.array(T)
_testlabel = np.array(C)

_testlabel

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [5]:
# 5. Declare data preprocessing steps
pipeline = make_pipeline(RandomForestClassifier())

# Add a dict of estimator and estimator related parameters in this list
hyperparameters = {
                'randomforestclassifier__n_estimators': [25,50,75,100],
                'randomforestclassifier__max_features' : [None, "log2", "auto"]
                }


In [6]:
# 7. Tune model using cross-validation pipeline
clf = GridSearchCV(pipeline, hyperparameters, cv=5,verbose=1,n_jobs=-1)
clf.fit(_traindata, _trainlabel)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 56.7min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 67.7min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_sam...bs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'randomforestclassifier__n_estimators': [25, 50, 75, 100], 'randomforestclassifier__max_features': [None, 'log2', 'auto']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [7]:
print(clf.best_params_)
print(clf.best_estimator_)
# print(clf.cv_results_ )

{'randomforestclassifier__max_features': 'log2', 'randomforestclassifier__n_estimators': 100}
Pipeline(memory=None,
     steps=[('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])


In [8]:
print(clf.best_score_ )

0.9985620550719694


In [9]:
print (clf.refit)
 
# 9. Evaluate model pipeline on test data
pred = clf.predict(_testdata)


from sklearn.metrics import accuracy_score
print(accuracy_score(_testlabel, pred))

True
0.9985680649656432


In [10]:
from sklearn.metrics import confusion_matrix,classification_report

cm = confusion_matrix(_testlabel, pred)
print(classification_report(_testlabel, pred))
print(cm)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    197610
           1       0.98      0.99      0.99     10500

   micro avg       1.00      1.00      1.00    208110
   macro avg       0.99      0.99      0.99    208110
weighted avg       1.00      1.00      1.00    208110

[[197430    180]
 [   118  10382]]


In [11]:
# 10. Save model for future use
joblib.dump(clf, 'rf_gridcv_tanpa_scaler_100-est-log2.pkl')

['rf_gridcv_tanpa_scaler_100-est-log2.pkl']

In [9]:
clf2 = joblib.load('v-rf_gridcv_robust.pkl')



In [10]:
print(clf2.best_params_)
print(clf2.best_estimator_)

{'randomforestclassifier__max_depth': None, 'randomforestclassifier__max_features': 'log2'}
Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='log2', max_leaf_nodes=None,
 ...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])
